# 앙상블

앙상블 기법이란 여러 개의 분류 모델을 조합해서 더 나은 성능을 내는 방법이다.


## 1. 배깅
배깅은 과대적합이 쉬운 모델에 상당히 적합한 앙상블이다.  
배깅은 한 가지 분류 모델을 여러 개 만들어서 서로 다른 학습 데이터로 학습시킨 후(부트스트랩),  
동일한 테스트 데이터에 대한 서로 다른 예측값들을 투표를 통해(어그레이팅) 가장 높은 예측 값으로 최종 결론을 내리는 앙상블 기법이다.  

### 1) 부트스트랩
부트스트랩이란 데이터를 조금은 편향되도록 샘플링하는 기법이다. 보통 의사결정 트리처럼 과대적합되기 쉬운 모델을 앙상블할 때 많이 사용한다.  
과대적합 모델이란 학습 데이터에 대한 분산은 높고 편향이 적은 모델이며,  
학습 시에는 정확도가 높게 나오지만 학습에 사용된 적이 없던 데이터의 분류는 정확도가 낮게 나오는 모델이다.  
보통 하이퍼파라미터 설정 없이 학습한 의사결정 트리가 쉽게 과대적합된다.  
부트스트랩은 데이터 샘플링 시 편향을 높임으로써 분산이 높은 모델의 과대적합 위험을 줄이는 효과를 준다.  


### 2) 어그리게이팅
어그리게티잉은 여러 분류 모델이 예측한 값들을 조합해서 하나의 결론을 도출하는 과정이다. 결론은 투표를 통해 결정한다.

#### - 하드보팅
하드보팅은 선거철에 하는 투표와 동일하다. 배깅에 포함된 K개의 분류 모델에서 최대 득표를 받은 예측값으로 결론을 도출한다.

#### - 소프트 보팅
하드보팅의 경우 각 분류 모델은 최고의 확률을 갖는 분류값만을 어그리게이팅할 때 리턴하는 반면,  
소프트보팅은 모든 분류값의 확률을 리턴한다. 
따라서 하드 보팅은 단순히 가장 많은 투표를 받은 분류값을 단순히 어그리게이팅의 결론으로 도출하는 반면,  
소프트보팅은 각 분류값별 확률을 더해준 값을 점수로 사용해 최대 점수를 가진 분류값을 결론으로 도출한다.  

### 3) 랜덤포레스트 = 의사결정트리 + 배깅
랜덤포레스트는 여러 의사결정 트리를 배깅해서 예측을 실행하는 모델이다. 배깅은 많은 의사결정 트리 모델의 개선을 이뤘고,
여러 개의 나무들이 모여있다는 개념에서 랜덤 포레스트라는 이름이 생겨났다.  
의사결정 트리에서는 최적의 특징으로 분기하는 반면,  
랜덤 포레스트는 각 노드에 주어진 데이터를 샘플링해서 일부 데이터를 제외한 채 최적의 특징을 찾아 트리를 분기한다.


## 2. 부스팅
배깅은 서로 다른 알고리즘에 기반한 여러 분류기를 병렬적으로 학습하는 반면, 부스팅은 동일한 알고리즘의 분류기를 순차적으로 학습해서 여러 개의 분류기를 만든 후, 테스트 할 때 가중 투표를 통해 예측값을 결정한다.  
순차적 학습과 가중 투표가 부스팅의 가장 큰 특징이다.

### 1) 순차적 학습
순차적으로 학습 데이터를 보강하며 동일한 알고리즘의 분류기를 여러 개 만드는 과정을 거친다

### 2) 가중 투표
배깅은 마치 선거를 하듯, 동일한 한 표씩 부여되는 반면 부스팅은 가중 투표가 진행된다. 가중 투표는 투표자의 능력치에 따라 한표의 가치가 다른 투표이다.

# 랜덤 포레스트 손글씨 분류
MNIST 손글씨 데이터 셋을 사용해 랜덤 포레스트 모델과 의사결정 트리 모델을 동일하게 학습시켜서 두 모델의 성능 차이를 비교해본다.

In [1]:
from sklearn import datasets
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### 손글씨 데이터 불러오기

In [2]:
mnist = datasets.load_digits()
features, labels = mnist.data, mnist.target

### 교차 검증
교차검증을 10번 수행하여, 10번의 교차 검증 평균 정확도를 저장한다.

In [3]:
def cross_validation(classifier, features, labels):
    cv_scores = []
    
    for i in range(10):
        scores = cross_val_score(classifier, features, labels, cv=10, scoring='accuracy')
        cv_scores.append(scores.mean())
    
    return cv_scores

In [4]:
# 의사 결정 트리의 MNIST 손글씨 검증 정확도를 저장한다
dt_cv_scores = cross_validation(tree.DecisionTreeClassifier(), features, labels)

In [ ]:
# 랜덤 포레스트의 MNIST 손글씨 검증 정확도를 저장한다
rf_cv_scores = cross_validation(RandomForestClassifier(), features, labels)

### 랜덤포레스트 VS 의사결정트리 시각화

In [ ]:
cv_list = [
            ['random_forest',rf_cv_scores],
            ['decision_tree',dt_cv_scores],
    ]
df = pd.DataFrame.from_dict(dict(cv_list))

In [ ]:
df.plot()

시각화를 통해 확연한 성능 차이를 확인할 수 있다.  
10번의 검증 결과 모두 랜덤 포레스트가 별도의 하이퍼파라미터 설정이 없는 의사 결정 트리보다 월등히 높다.

### 의사결정트리 정확도

In [ ]:
np.mean(dt_cv_scores)

### 랜덤포레스트 정확도

In [ ]:
np.mean(rf_cv_scores)